In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import seaborn as sns
from fuzzywuzzy import fuzz, process
from sklearn.preprocessing import LabelEncoder
import nltk

pd.set_option('display.max_rows', 300)
plt.style.use('seaborn-v0_8')

/home/guilherme_gauss/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
### Carregar dataset

df = pd.read_csv('Salary_Data_Based_country_and_race.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df = df.rename(columns={'Education Level': 'Education_Level', 
                                    'Job Title': 'Job_Title', 
                                    'Years of Experience': 'Years_of_Experience'})
print(df.shape)

(6704, 8)


In [3]:
df.tail(3)

,Age,Gender,Education_Level,Job_Title,Years_of_Experience,Salary,Country,Race
6701,30.0,Female,Bachelor's Degree,Financial Manager,4.0,55000.0,China,Chinese
6702,46.0,Male,Master's Degree,Marketing Manager,14.0,140000.0,China,Korean
6703,26.0,Female,High School,Sales Executive,1.0,35000.0,Canada,Black


In [4]:
### para facilitar o trabalho, faço a separação das colunas numéricas e categóricas

d_col = dict()
d_col['num'] = ['Age', 'Years_of_Experience', 'Salary']
d_col['cat'] = ['Gender', 'Education_Level', 'Job_Title', 'Country', 'Race']

In [5]:
d_col

{'num': ['Age', 'Years_of_Experience', 'Salary'],
 'cat': ['Gender', 'Education_Level', 'Job_Title', 'Country', 'Race']}

In [6]:
### Feature Engineering

def replace_class(df, column, dict_replace):

    df[column] = df[column].replace(dict_replace)
    
    return df

def drop_class(df, column, class_):

    df = df[df[column]!=class_]
    
    return df

def clear_outliers(df, column, p):

    min_ = df[column].quantile(p)
    max_ = df[column].quantile(1-p)

    df = df[(df[column]>=min_) & (df[column]<=max_)]

    return df

def drop_column(df, column):

    df = df.drop(column, axis=1)

    return df

def categorical_encoding(df, ls_columns, datapath):
    
    le = LabelEncoder()

    for col in ls_columns:
            df[col] = le.fit_transform(df[col])
            le_json = dict(zip(le.classes_, le.transform(le.classes_).astype('str')))
            jsonpath = os.path.join(datapath, col+'.json')
            with open(jsonpath, 'w') as f:
                json.dump(le_json, f)
    
    return df

In [7]:

###################################################################################################################################################
#############################################------------------ Preprocessamento v1 ------------------#############################################
###################################################################################################################################################

def process_job_v1(df, datapath):

    ### Como o data_explore mostrou, existem uma boa parte do 'Job_Title' que aparecem com baixa frequencia e 
    ### que devem ser trabalhados pois não tem representação significativa em termo de machine learning.
    ### Atraves das distancias de strings, será feita um pareamento dos 'Job_Title' que aparecem menos de 3 vezes 
    ### com o restante, de modo que serão considerados equivalentes os jobs com pontuação maior que 90, 
    ### do contrário será excluido do dataset.
    
    job_vc = df['Job_Title'].value_counts()

    n = 5
    major_job = job_vc[job_vc>n].index ### Job_Title com frequencia maior que n
    minor_job = job_vc[job_vc<=n].index ### Job_Title com frequencia menor ou igual que n
    ls1 = []
    for mi in minor_job:
        ls2 = []
        for ma in major_job:
            ls2.append(fuzz.token_set_ratio(mi.replace('_', ' '), ma.replace('_', ' ')))
        ls1.append(ls2)

    ls1 = np.array(ls1)

    a = ls1.max(axis=1)
    b = major_job[ls1.argmax(axis=1)]

    df_job = pd.DataFrame({'job_minor': minor_job, 'score': a, 'job_major': b})

    ### Jobs minoritários que serão substituidos pelo Job com pontuação maior que 90
    df_rep = df_job[df_job['score']>=90]

    ### Jobs minoritários que serão removidos por não haver outro Job com pareamento suficiente.
    df_excl = df_job[df_job['score']<90]

    df = df[~df['Job_Title'].isin(df_excl['job_minor'])]

    for i, j in df_rep[['job_minor', 'job_major']].values:
        df['Job_Title'] = df['Job_Title'].replace(i, j)

    le = LabelEncoder()

    df['Job_Title'] = le.fit_transform(df['Job_Title'])
    le_json = dict(zip(le.classes_, le.transform(le.classes_).astype('str')))
    jsonpath = os.path.join(datapath, 'Job_Title.json')
    with open(jsonpath, 'w') as f:
        json.dump(le_json, f)
    columns_select = ['Gender', 'Education_Level', 'Years_of_Experience', 'Country', 'Race', 'Job_Title',
                    'Salary']

    return df[columns_select]

###################################################################################################################################################
#########---------------------------------------------------------------------------------------------------------------------------------#########
###################################################################################################################################################

###################################################################################################################################################
#############################################------------------ Preprocessamento v2 ------------------#############################################
###################################################################################################################################################
def process_job_v2(df, datapath):

    ### encoding Job_Title
    ### existe 192 classes de Job_Title, muito deles com frequencia única
    ### Algumas profissões são nomeadas com o nivel junior, senior, diretor etc. Porém há classes sem a especificidade do nivel
    ### Por exemplo, há "data_scientist", "senior_data_scientist", "junior_data_scientist" e "director_of_data_science"
    ### O tokenization e encoding será feito de forma que essas classes equivalentes a "data_scientist" sejam aproximadas, como o exemplo:
    ### "data_scientist": [0, 14, 20], "senior_data_scientist": [1, 14, 20], "junior_data_scientist": [3, 14, 20], "director_of_data_science": [6, 14, 20]
    ### Essa aproximação é feita pelos elementos em comum [14, 20], o que não ocorreria no caso de uma simples troca das classes por um número como
    ### "data_scientist": 1, "senior_data_scientist": 2, "junior_data_scientist": 3, "director_of_data_science": 4
    

    ls_word = np.concatenate(df['Job_Title'].drop_duplicates().reset_index(drop=True).str.split('_'))
    allWordDist = nltk.FreqDist(ls_word)
    allWordDist.most_common(25)

    n_vocab = len(allWordDist)
    d_vocab = {}
    d_vocab_r = {}
    for i, word in enumerate(list(allWordDist)):
        if word not in ['of', 'and']:
            d_vocab[word] = i+1
            d_vocab_r[i+1] = word
            
    jsonpath = os.path.join(datapath, 'Job_Title_words.json')
    with open(jsonpath, 'w') as f:
        json.dump(d_vocab, f)

    ls_vec = []
    for tokeniz in df['Job_Title'].str.split('_'):
        ls_vec.append([0, 0, 0, 0] + [d_vocab[w] for w in tokeniz if w in d_vocab.keys()])

    df['Job_Title_Vec'] = ls_vec
    df['Job_Title_Vec'] = df['Job_Title_Vec'].str.slice(-5)


    df['Job_Title_Vec_1'] = df['Job_Title_Vec'].apply(lambda x: x[0])
    df['Job_Title_Vec_2'] = df['Job_Title_Vec'].apply(lambda x: x[1])
    df['Job_Title_Vec_3'] = df['Job_Title_Vec'].apply(lambda x: x[2])
    df['Job_Title_Vec_4'] = df['Job_Title_Vec'].apply(lambda x: x[3])
    df['Job_Title_Vec_5'] = df['Job_Title_Vec'].apply(lambda x: x[4])
    df = df.dropna().reset_index(drop=True)

    columns_select = ['Gender', 'Education_Level', 'Years_of_Experience', 'Country', 'Race', 
                    'Job_Title_Vec_1', 'Job_Title_Vec_2', 'Job_Title_Vec_3', 'Job_Title_Vec_4', 'Job_Title_Vec_5',  
                    'Salary']

    return df[columns_select]

Esquema do processo de preparação da base

!['çiçhiokhç'](static/processa_dados.png)


In [8]:
df_preprocess = df.copy()

In [9]:
## São apenas 6 linhas do total de 6704, os missing serão excluidos pois não influenciará no resultado final
df_preprocess.dropna(inplace=True)

### adequação das classes 'Education_Level' e 'Race' de acordo com as exploração dos dados
df_preprocess = replace_class(df_preprocess, 'Education_Level', {"bachelor's_degree": "bachelor's", "master's_degree": "master's"})
df_preprocess = replace_class(df_preprocess, 'Race', {'australian': 'white', 'welsh': 'white', 'korean': 'asian', 'chinese': 'asian', 'african_american': 'black'})

### remoção dos gêneros 'other', dado sua insignificancia numérica
df_preprocess = drop_class(df_preprocess, 'Race', 'other')

### remover os dois extremos salariais
df_preprocess = clear_outliers(df_preprocess, 'Salary', .005)

### remover coluna
df_preprocess = drop_column(df_preprocess, 'Age')

### encoding categorical (o 'Job_Title' terá um tratamento diferente dado sua grande quantidade de classes)
ls_columns = ['Gender', 'Education_Level', 'Country', 'Race']
datapath = 'data_preprocess'
df_preprocess = categorical_encoding(df_preprocess, ls_columns, datapath)

In [10]:
### preprocessamento v1 do 'Job_Title' 
df_preprocess_v1 = process_job_v1(df_preprocess, datapath)

/tmp/ipykernel_10566/1400598623.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Job_Title'] = df['Job_Title'].replace(i, j)
/tmp/ipykernel_10566/1400598623.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Job_Title'] = le.fit_transform(df['Job_Title'])


In [11]:
filename = os.path.join(datapath, 'salary_preprocess_v1.csv')
df_preprocess_v1.to_csv(filename, index=None)

In [12]:
### preprocessamento v2 do 'Job_Title' 
df_preprocess_v2 = process_job_v2(df_preprocess, datapath)

In [13]:
filename = os.path.join(datapath, 'salary_preprocess_v2.csv')
df_preprocess_v2.to_csv(filename, index=None)